In [1]:
# Importamos las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Definimos un bloque residual (bloque de identidad)
def identity_block(x, filters):
    """
    Implementa un bloque de identidad residual.
    Args:
    - x: entrada al bloque.
    - filters: número de filtros en las capas convolucionales.
    """
    # Guardamos la entrada original para la conexión residual
    shortcut = x

    # Primera convolución
    x = Conv2D(filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Segunda convolución
    x = Conv2D(filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    # Suma de la entrada original (shortcut) con la salida
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

# Definimos un bloque convolucional (cambia el tamaño de entrada)
def convolutional_block(x, filters, strides=2):
    """
    Implementa un bloque convolucional residual que cambia el tamaño.
    Args:
    - x: entrada al bloque.
    - filters: número de filtros en las capas convolucionales.
    - strides: tamaño del paso en la primera convolución.
    """
    # Guardamos la entrada original para la conexión residual
    shortcut = Conv2D(filters, (1, 1), strides=strides, kernel_initializer='he_normal')(x)
    shortcut = BatchNormalization()(shortcut)

    # Primera convolución con cambio de tamaño
    x = Conv2D(filters, (3, 3), strides=strides, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Segunda convolución
    x = Conv2D(filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    # Suma de la entrada original (shortcut) con la salida
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

# Construimos la arquitectura ResNet-34
def ResNet34(input_shape, num_classes):
    """
    Construye la arquitectura ResNet-34.
    Args:
    - input_shape: forma de la entrada (alto, ancho, canales).
    - num_classes: número de clases para la salida.
    """
    # Capa de entrada
    inputs = Input(shape=input_shape)

    # Primera capa convolucional
    x = Conv2D(64, (7, 7), strides=2, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MaxPooling para reducir tamaño
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    # Bloques residuales
    # Bloques con 64 filtros
    for _ in range(3):
        x = identity_block(x, 64)

    # Bloques con 128 filtros
    x = convolutional_block(x, 128)
    for _ in range(3):
        x = identity_block(x, 128)

    # Bloques con 256 filtros
    x = convolutional_block(x, 256)
    for _ in range(5):
        x = identity_block(x, 256)

    # Bloques con 512 filtros
    x = convolutional_block(x, 512)
    for _ in range(2):
        x = identity_block(x, 512)

    # Capa de pooling global y salida
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Creamos el modelo
    model = Model(inputs, outputs, name='ResNet34')
    return model

# Configuración del modelo
input_shape = (224, 224, 3)  # Tamaño de las imágenes de entrada
num_classes = 1000  # Ejemplo para ImageNet con 1000 clases
model = ResNet34(input_shape, num_classes)

# Compilación del modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# NOTA: Entrenamiento y evaluación
# Para entrenar el modelo, usa model.fit() con tus datos preprocesados.
# Ejemplo:
# model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))


Model: "ResNet34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 112, 112, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 112, 112, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 112, 112, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 56, 56, 64)     │              0 │ re_lu[0][0]            │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 56, 56, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 56, 56, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 56, 56, 64)     │              0 │ max_pooling2d[0][0],   │
│                           │                        │                │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 56, 56, 64)     │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 56, 56, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4

 Total params: 21,823,208 (83.25 MB)

 Trainable params: 21,806,184 (83.18 MB)

 Non-trainable params: 17,024 (66.50 KB)